**EDA**


**Imports**


In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS, TfidfVectorizer
from datetime import datetime
from dateutil.parser import parse 
from google.colab import drive
drive.mount('/content/gdrive')


/usr/local/lib/python3.7/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


Mounted at /content/gdrive


**Load Data**

In [2]:
boulderfiretweet= "/content/gdrive/MyDrive/Natural Disaster Alert System/boulder_fire_tweets.csv"
bf = pd.read_csv(boulderfiretweet)
bf.head()

,ImberJane,2022-03-28 22:56:22+00:00,RT @oliviaprentzel: Mandatory evacuation orders were issued for a wildfire just east of Estes Park... two days after the #NCAR fire sparked…,1508578772563501058
0,boulder_fire,2022-03-28 22:55:38+00:00,RT @BoulderOEM: Containment of #NCARFire is cu...,1508578590010621955
1,RachelAnneLevy,2022-03-28 22:47:19+00:00,Climate change is dangerous and costly. \n\nWe...,1508576497744891908
2,Boulderchick,2022-03-28 22:45:07+00:00,No more Fire!! #Boulder this up near Estes ht...,1508575942947467264
3,KhalaCompany,2022-03-28 22:40:19+00:00,"RT @rmpbs: ""The concern is that any time there...",1508574735940128778
4,KhalaCompany,2022-03-28 22:39:29+00:00,RT @CUBoulderENVS: As we return from Spring Br...,1508574525851672588


**Rename Column**


In [3]:
bf = bf.rename(columns={"ImberJane": "name", "2022-03-28 22:56:22+00:00" : "date","RT @oliviaprentzel: Mandatory evacuation orders were issued for a wildfire just east of Estes Park... two days after the #NCAR fire sparked…": "text","1508578772563501058":"id"})


**Feature Engineering**

**Binarizing**

In [4]:
bf['word_count'] = bf['text'].apply(lambda x: len(x.split()))
bf['character_count'] = bf['text'].str.len()
bf['mention_count'] = bf['text'].apply(lambda x: str.count(x, '@'))

In [5]:
bf['mention'] = np.where(bf['text'].str.contains('@'), 1, 0)
bf['https'] = np.where(bf['text'].str.contains('http'), 1,0)
bf['CNN_mention'] = np.where(bf['text'].str.contains('@CNN'), 1, 0)
bf['bouldercolorado'] = np.where(bf['text'].str.contains('@bouldercolorado'), 1, 0)
bf['BeeAsMarine'] = np.where(bf['text'].str.contains('@BeeAsMarine'), 1, 0)
bf['boulder_fire'] = np.where(bf['text'].str.contains('@boulder_fire'), 1, 0)
bf.head()

,name,date,text,id,word_count,character_count,mention_count,mention,https,CNN_mention,bouldercolorado,BeeAsMarine,boulder_fire
0,boulder_fire,2022-03-28 22:55:38+00:00,RT @BoulderOEM: Containment of #NCARFire is cu...,1508578590010621955,24,144,1,1,0,0,0,0,0
1,RachelAnneLevy,2022-03-28 22:47:19+00:00,Climate change is dangerous and costly. \n\nWe...,1508576497744891908,21,140,0,0,1,0,0,0,0
2,Boulderchick,2022-03-28 22:45:07+00:00,No more Fire!! #Boulder this up near Estes ht...,1508575942947467264,9,67,0,0,1,0,0,0,0
3,KhalaCompany,2022-03-28 22:40:19+00:00,"RT @rmpbs: ""The concern is that any time there...",1508574735940128778,23,139,1,1,1,0,0,0,0
4,KhalaCompany,2022-03-28 22:39:29+00:00,RT @CUBoulderENVS: As we return from Spring Br...,1508574525851672588,21,139,2,1,0,0,0,0,0


In [6]:
bf['boulder_fire'].value_counts()

0    4651
1     348
Name: boulder_fire, dtype: int64

In [7]:
bf['bouldercolorado'].value_counts()

0    4919
1      80
Name: bouldercolorado, dtype: int64

**Sentiment Analysis**